In [16]:
!cd .. && scripts/build_and_push.sh

scripts/build_and_push.sh: line 1: ·!/bin/bash: No such file or directory
ECR Login
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image
Sending build context to Docker daemon  366.6kB
Step 1/9 : FROM nvcr.io/nvidia/pytorch:20.08-py3
 ---> c710aa2340b4
Step 2/9 : ENV PYTHONDONTWRITEBYTECODE=1
 ---> Using cache
 ---> 8b9d73b4d356
Step 3/9 : ENV PYTHONUNBUFFERED=1
 ---> Using cache
 ---> 2053c99d2b7b
Step 4/9 : RUN apt-get update && apt-get install -y --no-install-recommends nginx curl
 ---> Using cache
 ---> a43c5e2ca24a
Step 5/9 : WORKDIR /opt/ml/
 ---> Using cache
 ---> de16effb574a
Step 6/9 : RUN pip install sagemaker-training
 ---> Using cache
 ---> 78e1af9dbdaa
Step 7/9 : COPY src/ /opt/ml/code
 ---> Using cache
 ---> 5de330c563a7
Step 8/9 : ENV SAGEMAKER_PROGRAM train
 ---> Using 

In [2]:
from sagemaker.session import get_execution_role, Session
import os

sagemaker_role = get_execution_role()
sagemaker_session = Session()

In [35]:
s3_uri_data = 's3://sagemaker-us-west-2-256305374409/TiendaApp/data/'


In [37]:
!aws s3 ls $s3_uri
!aws ecr describe-repositories --registry-id $account

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-west-2:430127992102:repository/yolo_train",
            "registryId": "430127992102",
            "repositoryName": "yolo_train",
            "repositoryUri": "430127992102.dkr.ecr.us-west-2.amazonaws.com/yolo_train",
            "createdAt": 1599690799.0,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        }
    ]
}


In [7]:
!aws s3 pwd

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument subcommand: Invalid choice, valid choices are:

ls                                       | website                                 
cp                                       | mv                                      
rm                                       | sync                                    
mb                                       | rb                                      
presign                                 


In [9]:
from time import gmtime, strftime

bucket_name = sagemaker_session.default_bucket()
key_name = "TiendaApp"
s3_uri_data = "s3://{}/{}/{}/".format(bucket_name, key_name, "data")
s3_uri_output = "s3://{}/{}/{}/".format(bucket_name, key_name, "model")
account = sagemaker_session.boto_session.client('sts').get_caller_identity()['Account']
region = sagemaker_session.boto_session.region_name
image_name = "yolo_train"
image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}".format(account, region, image_name)
#image_uri = "registry.hub.docker.com/repository/docker/cvillad/{0}".format(image_name)
base_job_name = "test-training-job-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
os.environ["account"] = account
os.environ["s3_uri"] = s3_uri_data
print("Training Job name : {}".format(base_job_name))
print("S3 uri input: {}".format(s3_uri_data))
print("S3 uri output: {}".format(s3_uri_output))
print("image uri: {}".format(image_uri))
!aws s3 ls $s3_uri
!aws ecr describe-repositories --registry-id $account

Training Job name : test-training-job-2020-09-10-15-00-26
S3 uri input: s3://sagemaker-us-west-2-430127992102/TiendaApp/data/
S3 uri output: s3://sagemaker-us-west-2-430127992102/TiendaApp/model/
image uri: 430127992102.dkr.ecr.us-west-2.amazonaws.com/yolo_train
                           PRE training/
                           PRE validating/
2020-09-10 14:51:32         80 obj.data
2020-09-10 14:51:32        532 obj.names
2020-09-10 14:51:32      50550 train.txt
2020-09-10 14:51:32      23436 val.txt
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-west-2:430127992102:repository/yolo_train",
            "registryId": "430127992102",
            "repositoryName": "yolo_train",
            "repositoryUri": "430127992102.dkr.ecr.us-west-2.amazonaws.com/yolo_train",
            "createdAt": 1599690799.0,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "enc

In [10]:
from sagemaker.estimator import Estimator

yolov3_estimator = Estimator(image_name=image_uri, 
                            role=sagemaker_role,
                            train_instance_count=1,
                            train_instance_type="ml.m5.large",
                            train_volume_size=35,
                            sagemaker_session=sagemaker_session,
                            base_job_name = base_job_name,
                            hyperparameters={"test": "this is a test", "batch": 32},
                            tags=[{"Key": "Name", "Value": "test-job"},
                                 {"Key": "Description", "Value": "Test training job"}])


yolov3_estimator.fit(inputs={"yolo_data": s3_uri_data}, job_name=base_job_name, wait=True)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-09-10 15:00:30 Starting - Starting the training job...
2020-09-10 15:00:32 Starting - Launching requested ML instances......
2020-09-10 15:01:46 Starting - Preparing the instances for training...
2020-09-10 15:02:20 Downloading - Downloading input data......
2020-09-10 15:03:13 Training - Downloading the training image..........................=============
== PyTorch ==

NVIDIA Release 20.08 (build 15516749)
PyTorch Version 1.7.0a0+8deb4fe

Container image Copyright (c) 2020, NVIDIA CORPORATION.  All rights reserved.

Copyright (c) 2014-2020 Facebook Inc.
Copyright (c) 2011-2014 Idiap Research Institute (Ronan Collobert)
Copyright (c) 2012-2014 Deepmind Technologies    (Koray Kavukcuoglu)
Copyright (c) 2011-2012 NEC Laboratories America (Koray Kavukcuoglu)
Copyright (c) 2011-2013 NYU                      (Clement Farabet)
Copyright (c) 2006-2010 NEC Laboratories America (Ronan Collobert, Leon Bottou, Iain Melvin, Jason Weston)
Copyright (c) 2006      Idiap Research Institute (Sam